In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn import model_selection,preprocessing,ensemble

data_df=pd.read_csv("train.csv")

/home/admin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
def feature1(data, test=False):
    if test == 1:
        dealed_data = data.drop_duplicates(['file_id'])['file_id']
    else:
        dealed_data = data.drop_duplicates(['file_id'])[['file_id','label']]
    # 提取各类别特征的类别数
    dealed_train = data.groupby("file_id")[["api",'tid','return_value']].nunique()
    dealed_train.columns = ['api_nunique','tid_nunique','value_nunique']
    
    temp = data.groupby(['file_id'])['index'].count().rename('id_count')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 对每个file_id,的每个feat计数，然后求其统计特征
    cate_feat = ['api','tid','return_value']
    for feat in cate_feat:
        temp = data.groupby(['file_id',feat])[feat].count().groupby(['file_id']).agg(['min','max','mean','median','std',pd.Series.mad,
                                                                            pd.Series.skew,pd.Series.kurt]).add_prefix(feat+'_cnt_')
        dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 提取交叉特征，每个file_id，每个不同api的tid种类数的统计特征
    temp = data.groupby(['file_id','api'])['tid'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('api_tid_')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 提取交叉特征，每个file_id，每个不同api的return种类数的统计特征
    temp = data.groupby(['file_id','api'])['return_value'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('api_value_')                                                               
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    temp = data.groupby(['file_id','tid'])['api'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('tid_tid_')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    temp = data.groupby(['file_id','tid'])['return_value'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('tid_value_')                                                               
    dealed_train = pd.concat([dealed_train, temp],axis=1)
    return dealed_train

train_data = feature1(data_df, test=False)
#test_data = feature1(test, test=True)
train_data.to_csv('train_featur_all.csv', index=False)
#test_data.to_csv('test_feature1.csv', index=False)

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
api_text=data_df.groupby(['file_id'])['api'].apply(lambda x:' '.join(list(x))).to_frame().reset_index()

In [ ]:
train_data=pd.merge(train_data,api_text,how='left', on='file_id')

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
train_X=train_data.iloc[:,:-1]
train_Y=data_df.groupby(['file_id','label'])['label'].unique()
train_Y=np.array(train_y)

In [ ]:
def lgb_logloss(preds,data):
    labels_ = data.get_label()
    classes_ = np.unique(labels_) 
    preds_prob = []
    
    for i in range(len(classes_)):
        preds_prob.append(preds[i*len(labels_):(i+1) * len(labels_)])
    #print(preds_prob)
    preds_prob_ = np.vstack(preds_prob) 
    
    loss = [] 
    for i in range(preds_prob_.shape[1]):  # 样本个数
        sum_ = 0  
        for j in range(preds_prob_.shape[0]): #类别个数
            pred = preds_prob_[j,i] # 第i个样本预测为第j类的概率
            if  j == labels_[i]:
                sum_ += np.log(pred)
            else:
                sum_ += np.log(1 - pred) 
             
        loss.append(sum_)  
         
    return 'loss is: ' ,-1 * (np.sum(loss) / preds_prob_.shape[1]),False

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_Y, test_size=0.2, random_state=0)

In [ ]:
def runLGB(x_train,y_train,x_valid,y_valid):
    dtrain = lgb.Dataset(x_train,y_train) 
    dval   = lgb.Dataset(x_valid,y_valid, reference = dtrain) 

    params = {
         'task':'train', 
         'num_leaves': 255,
         'objective': 'multiclass',
         'num_class':6,
        #'min_data_in_leaf': 40,
         'min_data_in_leaf': 1,
        'learning_rate': 0.05,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.9,
         'bagging_freq': 5, 
         'max_bin':128,
        'num_threads': 10,
        'random_state':100
     }  
    lgb_model_0_order = lgb.train(params, dtrain, num_boost_round=500,valid_sets=[dtrain,dval], early_stopping_rounds=50)  

In [ ]:
runLGB(x_train,y_train,x_valid,y_valid)

In [18]:
from sklearn.feature_extraction.text  import TfidfVectorizer
from scipy import sparse
import os 
import pickle
n_range=(1,3)  #
max_feature=100000 #

if os.path.exists("api_tfidf.pkl")==False:
    api_tfidf=TfidfVectorizer(ngram_range=n_range,max_features=max_feature)
    api_tfidf.fit(train_data['api'])
    with open("api_tfidf.pkl",'wb') as f:
        pickle.dump(api_tfidf,f)
        
else:
    with open("api_tfidf.pkl",'rb')  as f:
        api_tfidf=pickle.load(f)




In [19]:
tr_sparse1=api_tfidf.transform(train_data['api'].values)

In [20]:
train_data.iloc[:,:-1].shape,tr_sparse1.shape

((116624, 61), (116624, 66540))

In [21]:
train_X=tr_sparse1

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50805	test-mlogloss:1.50837
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29799	test-mlogloss:1.29853
[2]	train-mlogloss:1.13258	test-mlogloss:1.13338
[3]	train-mlogloss:0.9967	test-mlogloss:0.997618
[4]	train-mlogloss:0.882631	test-mlogloss:0.883652
[5]	train-mlogloss:0.785398	test-mlogloss:0.78662
[6]	train-mlogloss:0.701508	test-mlogloss:0.702931
[7]	train-mlogloss:0.628052	test-mlogloss:0.629634
[8]	train-mlogloss:0.56374	test-mlogloss:0.565399
[9]	train-mlogloss:0.506943	test-mlogloss:0.508717
[10]	train-mlogloss:0.456703	test-mlogloss:0.458619
[11]	train-mlogloss:0.412078	test-mlogloss:0.414033
[12]	train-mlogloss:0.372285	test-mlogloss:0.374431
[13]	train-mlogloss:0.336921	test-mlogloss:0.33919
[14]	train-mlogloss:0.305016	test-mlogloss:0.307391
[15]	train-mlogloss:0.276481	test-mlogloss:0.279012
[16]	train-mlogloss:0.250891	test-mlog

In [23]:
from sklearn.feature_extraction.text  import TfidfVectorizer
from scipy import sparse
import os 
import pickle
n_range=(1,2)  #
max_feature=100000 #

if os.path.exists("api_tfidf_bi.pkl")==False:
    api_tfidf=TfidfVectorizer(ngram_range=n_range,max_features=max_feature)
    api_tfidf.fit(train_data['api'])
    with open("api_tfidf_bi.pkl",'wb') as f:
        pickle.dump(api_tfidf,f)
        
else:
    with open("api_tfidf_bi.pkl",'rb')  as f:
        api_tfidf=pickle.load(f)




In [24]:
tr_sparse2=api_tfidf.transform(train_data['api'].values)

In [25]:
train_data.iloc[:,:-1].shape,tr_sparse2.shape

((116624, 61), (116624, 30984))

In [26]:
train_X=tr_sparse2

In [27]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50851	test-mlogloss:1.50905
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29892	test-mlogloss:1.29988
[2]	train-mlogloss:1.13528	test-mlogloss:1.13657
[3]	train-mlogloss:1.00015	test-mlogloss:1.00159
[4]	train-mlogloss:0.886164	test-mlogloss:0.888004
[5]	train-mlogloss:0.788786	test-mlogloss:0.790848
[6]	train-mlogloss:0.704951	test-mlogloss:0.707264
[7]	train-mlogloss:0.632093	test-mlogloss:0.634606
[8]	train-mlogloss:0.567653	test-mlogloss:0.570403
[9]	train-mlogloss:0.510799	test-mlogloss:0.51371
[10]	train-mlogloss:0.460909	test-mlogloss:0.464047
[11]	train-mlogloss:0.416631	test-mlogloss:0.419938
[12]	train-mlogloss:0.37666	test-mlogloss:0.38007
[13]	train-mlogloss:0.340862	test-mlogloss:0.344359
[14]	train-mlogloss:0.309351	test-mlogloss:0.313028
[15]	train-mlogloss:0.280934	test-mlogloss:0.284766
[16]	train-mlogloss:0.255221	test-mlog

In [1]:
from sklearn.feature_extraction.text  import TfidfVectorizer
from scipy import sparse
import os 
import pickle
n_range=(1,1)  #
max_feature=100000 #

if os.path.exists("api_tfidf_uni.pkl")==False:
    api_tfidf=TfidfVectorizer(ngram_range=n_range,max_features=max_feature)
    api_tfidf.fit(train_data['api'])
    with open("api_tfidf_uni.pkl",'wb') as f:
        pickle.dump(api_tfidf,f)
        
else:
    with open("api_tfidf_uni.pkl",'rb')  as f:
        api_tfidf=pickle.load(f)




In [2]:
tr_sparse3=api_tfidf.transform(train_data['api'].values)

NameError: name 'train_data' is not defined

In [1]:
train_data.iloc[:,:-1].shape,tr_sparse3.shape

NameError: name 'train_data' is not defined

In [31]:
train_X=tr_sparse3

In [32]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.51289	test-mlogloss:1.51329
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.30644	test-mlogloss:1.30732
[2]	train-mlogloss:1.14433	test-mlogloss:1.14557
[3]	train-mlogloss:1.01166	test-mlogloss:1.01318
[4]	train-mlogloss:0.899392	test-mlogloss:0.901356
[5]	train-mlogloss:0.804084	test-mlogloss:0.80635
[6]	train-mlogloss:0.720769	test-mlogloss:0.723242
[7]	train-mlogloss:0.648298	test-mlogloss:0.651071
[8]	train-mlogloss:0.585099	test-mlogloss:0.588312
[9]	train-mlogloss:0.529488	test-mlogloss:0.533075
[10]	train-mlogloss:0.479746	test-mlogloss:0.483658
[11]	train-mlogloss:0.435416	test-mlogloss:0.439556
[12]	train-mlogloss:0.39594	test-mlogloss:0.400302
[13]	train-mlogloss:0.360626	test-mlogloss:0.36525
[14]	train-mlogloss:0.328954	test-mlogloss:0.333861
[15]	train-mlogloss:0.300908	test-mlogloss:0.306052
[16]	train-mlogloss:0.275255	test-mlog

[155]	train-mlogloss:0.009039	test-mlogloss:0.030893
[156]	train-mlogloss:0.008966	test-mlogloss:0.03092
[157]	train-mlogloss:0.008897	test-mlogloss:0.030909
[158]	train-mlogloss:0.008843	test-mlogloss:0.030908
[159]	train-mlogloss:0.008792	test-mlogloss:0.030883
[160]	train-mlogloss:0.008729	test-mlogloss:0.03088
[161]	train-mlogloss:0.008678	test-mlogloss:0.03089
[162]	train-mlogloss:0.008643	test-mlogloss:0.030873
[163]	train-mlogloss:0.008595	test-mlogloss:0.030889
[164]	train-mlogloss:0.008542	test-mlogloss:0.030874
[165]	train-mlogloss:0.00848	test-mlogloss:0.030846
[166]	train-mlogloss:0.008441	test-mlogloss:0.030844
[167]	train-mlogloss:0.008389	test-mlogloss:0.030838
[168]	train-mlogloss:0.008349	test-mlogloss:0.030858
[169]	train-mlogloss:0.008308	test-mlogloss:0.030882
[170]	train-mlogloss:0.008251	test-mlogloss:0.030867
[171]	train-mlogloss:0.008194	test-mlogloss:0.030861
[172]	train-mlogloss:0.008125	test-mlogloss:0.030842
[173]	train-mlogloss:0.00808	test-mlogloss:0.03082

In [33]:
train_X = sparse.hstack([train_data.iloc[:,:-1],tr_sparse1]).tocsr()

In [34]:
train_X.shape,train_y.shape

((116624, 66601), (116624,))

In [35]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50806	test-mlogloss:1.50849
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29816	test-mlogloss:1.29873
[2]	train-mlogloss:1.13231	test-mlogloss:1.13304
[3]	train-mlogloss:0.99616	test-mlogloss:0.997115
[4]	train-mlogloss:0.882032	test-mlogloss:0.883126
[5]	train-mlogloss:0.785154	test-mlogloss:0.786455
[6]	train-mlogloss:0.701466	test-mlogloss:0.702882
[7]	train-mlogloss:0.628036	test-mlogloss:0.629566
[8]	train-mlogloss:0.563555	test-mlogloss:0.565206
[9]	train-mlogloss:0.506791	test-mlogloss:0.508607
[10]	train-mlogloss:0.45644	test-mlogloss:0.458398
[11]	train-mlogloss:0.411809	test-mlogloss:0.413924
[12]	train-mlogloss:0.371985	test-mlogloss:0.374213
[13]	train-mlogloss:0.336451	test-mlogloss:0.338828
[14]	train-mlogloss:0.304668	test-mlogloss:0.307161
[15]	train-mlogloss:0.276186	test-mlogloss:0.27887
[16]	train-mlogloss:0.250466	test-ml

In [36]:
train_X = sparse.hstack([train_data.iloc[:,:-1],tr_sparse2]).tocsr()
train_X.shape,train_y.shape

((116624, 31045), (116624,))

In [37]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50873	test-mlogloss:1.50928
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29926	test-mlogloss:1.30009
[2]	train-mlogloss:1.1336	test-mlogloss:1.13475
[3]	train-mlogloss:0.997892	test-mlogloss:0.999252
[4]	train-mlogloss:0.884058	test-mlogloss:0.885687
[5]	train-mlogloss:0.786565	test-mlogloss:0.78841
[6]	train-mlogloss:0.703069	test-mlogloss:0.70515
[7]	train-mlogloss:0.629733	test-mlogloss:0.632049
[8]	train-mlogloss:0.565425	test-mlogloss:0.56798
[9]	train-mlogloss:0.509215	test-mlogloss:0.512037
[10]	train-mlogloss:0.459003	test-mlogloss:0.462001
[11]	train-mlogloss:0.414798	test-mlogloss:0.418031
[12]	train-mlogloss:0.374799	test-mlogloss:0.378196
[13]	train-mlogloss:0.339603	test-mlogloss:0.343184
[14]	train-mlogloss:0.307574	test-mlogloss:0.311298
[15]	train-mlogloss:0.278859	test-mlogloss:0.28273
[16]	train-mlogloss:0.253093	test-mlog

In [38]:
train_X = sparse.hstack([train_data.iloc[:,:-1],tr_sparse3]).tocsr()
train_X.shape,train_y.shape

((116624, 369), (116624,))

In [39]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.51206	test-mlogloss:1.5126
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.30554	test-mlogloss:1.30661
[2]	train-mlogloss:1.1425	test-mlogloss:1.144
[3]	train-mlogloss:1.00876	test-mlogloss:1.01049
[4]	train-mlogloss:0.896565	test-mlogloss:0.89859
[5]	train-mlogloss:0.800499	test-mlogloss:0.802884
[6]	train-mlogloss:0.717618	test-mlogloss:0.720431
[7]	train-mlogloss:0.645337	test-mlogloss:0.648601
[8]	train-mlogloss:0.581583	test-mlogloss:0.585146
[9]	train-mlogloss:0.525202	test-mlogloss:0.528974
[10]	train-mlogloss:0.475117	test-mlogloss:0.47917
[11]	train-mlogloss:0.430497	test-mlogloss:0.434822
[12]	train-mlogloss:0.390687	test-mlogloss:0.395261
[13]	train-mlogloss:0.355104	test-mlogloss:0.359887
[14]	train-mlogloss:0.323217	test-mlogloss:0.328204
[15]	train-mlogloss:0.295079	test-mlogloss:0.300329
[16]	train-mlogloss:0.269408	test-mloglos

[155]	train-mlogloss:0.004248	test-mlogloss:0.027998
[156]	train-mlogloss:0.004219	test-mlogloss:0.028004
[157]	train-mlogloss:0.004176	test-mlogloss:0.027986
[158]	train-mlogloss:0.004129	test-mlogloss:0.027994
[159]	train-mlogloss:0.004098	test-mlogloss:0.028004
[160]	train-mlogloss:0.004059	test-mlogloss:0.027993
[161]	train-mlogloss:0.004022	test-mlogloss:0.028005
[162]	train-mlogloss:0.003983	test-mlogloss:0.028024
[163]	train-mlogloss:0.003945	test-mlogloss:0.028044
[164]	train-mlogloss:0.003895	test-mlogloss:0.028062
[165]	train-mlogloss:0.00385	test-mlogloss:0.02808
[166]	train-mlogloss:0.003809	test-mlogloss:0.02809
[167]	train-mlogloss:0.003764	test-mlogloss:0.028071
[168]	train-mlogloss:0.003721	test-mlogloss:0.028091
[169]	train-mlogloss:0.003687	test-mlogloss:0.028099
[170]	train-mlogloss:0.003647	test-mlogloss:0.028089
[171]	train-mlogloss:0.00361	test-mlogloss:0.028099
[172]	train-mlogloss:0.003578	test-mlogloss:0.028089
[173]	train-mlogloss:0.00355	test-mlogloss:0.02807

In [42]:
train_X = sparse.hstack([tr_sparse2,tr_sparse3]).tocsr()
train_X.shape,train_y.shape

((116624, 31292), (116624,))

In [43]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50802	test-mlogloss:1.50853
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29872	test-mlogloss:1.29963
[2]	train-mlogloss:1.13356	test-mlogloss:1.13484
[3]	train-mlogloss:0.998775	test-mlogloss:1.00031
[4]	train-mlogloss:0.885518	test-mlogloss:0.887339
[5]	train-mlogloss:0.788086	test-mlogloss:0.790138
[6]	train-mlogloss:0.703927	test-mlogloss:0.706178
[7]	train-mlogloss:0.630999	test-mlogloss:0.633506
[8]	train-mlogloss:0.56659	test-mlogloss:0.569283
[9]	train-mlogloss:0.50995	test-mlogloss:0.512797
[10]	train-mlogloss:0.459784	test-mlogloss:0.462826
[11]	train-mlogloss:0.415062	test-mlogloss:0.418248
[12]	train-mlogloss:0.375477	test-mlogloss:0.378901
[13]	train-mlogloss:0.3398	test-mlogloss:0.343425
[14]	train-mlogloss:0.307985	test-mlogloss:0.311742
[15]	train-mlogloss:0.279378	test-mlogloss:0.283228
[16]	train-mlogloss:0.254066	test-mlog

In [44]:
train_X = sparse.hstack([tr_sparse1,tr_sparse2,tr_sparse3]).tocsr()
train_X.shape,train_y.shape

((116624, 97832), (116624,))

In [45]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)

[0]	train-mlogloss:1.50795	test-mlogloss:1.50848
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.29805	test-mlogloss:1.29886
[2]	train-mlogloss:1.1319	test-mlogloss:1.1329
[3]	train-mlogloss:0.99606	test-mlogloss:0.997207
[4]	train-mlogloss:0.882086	test-mlogloss:0.883416
[5]	train-mlogloss:0.78464	test-mlogloss:0.786084
[6]	train-mlogloss:0.700412	test-mlogloss:0.701961
[7]	train-mlogloss:0.627189	test-mlogloss:0.62893
[8]	train-mlogloss:0.562819	test-mlogloss:0.564643
[9]	train-mlogloss:0.506157	test-mlogloss:0.50806
[10]	train-mlogloss:0.455973	test-mlogloss:0.457956
[11]	train-mlogloss:0.411391	test-mlogloss:0.413482
[12]	train-mlogloss:0.371699	test-mlogloss:0.373984
[13]	train-mlogloss:0.336233	test-mlogloss:0.338705
[14]	train-mlogloss:0.304453	test-mlogloss:0.307116
[15]	train-mlogloss:0.275939	test-mlogloss:0.278714
[16]	train-mlogloss:0.250372	test-mlogl

In [56]:
train_X = sparse.hstack([train_data.iloc[:,:-1],tr_sparse1,tr_sparse2,tr_sparse3]).tocsr()
train_X.shape,train_y.shape

((116624, 97893), (116624,))

In [57]:
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runLGB(x_train,y_train,x_valid,y_valid)